# Modules

In [ ]:
!pip install EMD-signal

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [55]:
import pickle
import numpy as np
import pandas as pd
import antropy as ant
from sklearn import svm
from sklearn import metrics
from scipy.signal import welch
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from PyEMD import EMD
from scipy.signal import butter, filtfilt
from scipy.linalg import svd
from sklearn.preprocessing import StandardScaler

# Data Preprocessing

In [2]:
a12 = pd.read_csv("../Dataset/as/ones/a12.csv").iloc[:4864, 1:9].dropna().to_numpy()
a13 = pd.read_csv("../Dataset/as/ones/a13.csv").iloc[:4864, 1:9].dropna().to_numpy()
a11 = pd.read_csv("../Dataset/as/ones/Letter_A.csv").iloc[:4864, 1:9].dropna().to_numpy()
a21 = pd.read_csv("../Dataset/as/twos/a21.csv").iloc[:4864, 1:9].dropna().to_numpy()
a4 = pd.read_csv("../Dataset/as/fours/a4.csv").iloc[:4864, 1:9].dropna().to_numpy()

as_1 = pd.read_csv("../Dataset/dataset/as_1.csv").dropna().iloc[:2304, 1:9].to_numpy()

ak_1 = pd.read_csv("../Dataset/A11_K.csv").dropna().iloc[0:12032, 1:9].to_numpy()
ak_2 = pd.read_csv("../Dataset/A12_K.csv").dropna().iloc[0:12032, 1:9].to_numpy()

In [3]:
b12 = pd.read_csv("../Dataset/bs/ones/b12.csv").dropna().iloc[:4864, 1:9].to_numpy()
b13 = pd.read_csv("../Dataset/bs/ones/b13.csv").dropna().iloc[:4864, 1:9].to_numpy()
b11 = pd.read_csv("../Dataset/bs/ones/Letter_B.csv").dropna().iloc[:4864, 1:9].to_numpy()
b31 = pd.read_csv("../Dataset/bs/threes/b31.csv").dropna().iloc[:4864, 1:9].to_numpy()
b4 = pd.read_csv("../Dataset/bs/fours/b4.csv").dropna().iloc[:4864, 1:9].to_numpy()

bs_1 = pd.read_csv("../Dataset/dataset/bs_1.csv").dropna().iloc[:2304, 1:9].to_numpy()

bk_1 = pd.read_csv("../Dataset/B11_K.csv").dropna().iloc[:12032, 1:9].to_numpy()
bk_2 = pd.read_csv("../Dataset/B12_K.csv").dropna().iloc[:12032, 1:9].to_numpy()



In [4]:
base1s = pd.read_csv("../Dataset/baselines/Baseline_EEG.csv").dropna().iloc[:, 1:9]
base2s = pd.read_csv("../Dataset/baselines/rest2.csv").dropna().iloc[:, 1:9]
base3s = pd.read_csv("../Dataset/baselines/rest3.csv").dropna().iloc[:, 1:9]
base4s = pd.read_csv("../Dataset/baselines/base4.csv").dropna().iloc[:, 1:9]


base1s_1 = pd.read_csv("../Dataset/dataset/rest_s1.csv").dropna().iloc[:, 1:9]

base1k_1 = pd.read_csv("../Dataset/rest_eeg_K.csv").dropna().iloc[:, 1:9]

In [5]:
mean_1 = base1s.describe().iloc[1].to_numpy().reshape((1, 8))
std_1 = base1s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_2 = base2s.describe().iloc[1].to_numpy().reshape((1, 8))
std_2 = base2s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_3 = base3s.describe().iloc[1].to_numpy().reshape((1, 8))
std_3 = base3s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_4 = base4s.describe().iloc[1].to_numpy().reshape((1, 8))
std_4 = base4s.describe().iloc[2].to_numpy().reshape((1, 8))

mean1_s = base1s_1.describe().iloc[1].to_numpy().reshape((1, 8))
std1_s = base1s_1.describe().iloc[2].to_numpy().reshape((1, 8))

mean1_k = base1k_1.describe().iloc[1].to_numpy().reshape((1, 8))
std1_k = base1k_1.describe().iloc[2].to_numpy().reshape((1, 8))

## BaseLine Correction

In [6]:
a12_new = np.zeros((19, 256, 8))
a13_new = np.zeros((19, 256, 8))
a11_new = np.zeros((19, 256, 8))
a21_new = np.zeros((19, 256, 8))
a4_new = np.zeros((19, 256, 8))
as_new = np.zeros((9, 256, 8))
ak1_new = np.zeros((46, 256, 8))
ak2_new = np.zeros((46, 256, 8))

b12_new = np.zeros((19, 256, 8))
b13_new = np.zeros((19, 256, 8))
b11_new = np.zeros((19, 256, 8))
b31_new = np.zeros((19, 256, 8))
b4_new = np.zeros((19, 256, 8))
bs_new = np.zeros((9, 256, 8))
bk1_new = np.zeros((46, 256, 8))
bk2_new = np.zeros((46, 256, 8))

for i in range(19):
    for j in range(256):
        for k in range(8):
            a12_new[i, j, k] = (a12[i+j, k] - mean_1[0, k])/std_1[0, k]
            a13_new[i, j, k] = (a13[i+j, k] - mean_1[0, k])/std_1[0, k]
            a11_new[i, j, k] = (a11[i+j, k] - mean_1[0, k])/std_1[0, k]
            a21_new[i, j, k] = (a21[i+j, k] - mean_2[0, k])/std_2[0, k]
            a4_new[i, j, k] = (a4[i+j, k] - mean_4[0, k])/std_4[0, k]



            b12_new[i, j, k] = (b12[i+j, k] - mean_1[0, k])/std_1[0, k]
            b13_new[i, j, k] = (b13[i+j, k] - mean_1[0, k])/std_1[0, k]
            b11_new[i, j, k] = (b11[i+j, k] - mean_1[0, k])/std_1[0, k]
            b31_new[i, j, k] = (b31[i+j, k] - mean_3[0, k])/std_3[0, k]
            b4_new[i, j, k] = (b4[i+j, k] - mean_4[0, k])/std_4[0, k]

for i in range(9):
    for j in range(256):
        for k in range(8):
            as_new[i, j, k] = (as_1[i+j, k] - mean1_s[0, k])/std1_s[0, k]
            bs_new[i, j, k] = (bs_1[i+j, k] - mean1_s[0, k])/std1_s[0, k]


for i in range(46):
    for j in range(256):
        for k in range(8):
            ak1_new[i, j, k] = (ak_1[i+j, k] - mean1_k[0, k])/std1_k[0, k]
            ak2_new[i, j, k] = (ak_2[i+j, k] - mean1_k[0, k])/std1_k[0, k]

            bk1_new[i, j, k] = (bk_1[i+j, k] - mean1_k[0, k])/std1_k[0, k]
            bk2_new[i, j, k] = (bk_2[i+j, k] - mean1_k[0, k])/std1_k[0, k]

In [100]:
all_a_s = np.concatenate((a12_new, a13_new, a11_new, a21_new, a4_new, as_new, ak1_new, ak2_new), axis = 0)
all_b_s = np.concatenate((b12_new, b13_new, b11_new, b31_new, b4_new, bs_new, bk1_new, bk2_new), axis = 0)

all_a_s = all_a_s.reshape((196, 256, 8))
all_b_s = all_b_s.reshape((196, 256, 8))

## EMD

In [179]:
def emd_signals(input_data):
    imf_count = 7
    emd = EMD()
    #input data is of the size: (196,256,8)
    #For EMD, the input data will be of the form: (trails, time_points, channels)
    #We get the data info first:
    trails, time_points, channels = input_data.shape
    imfs = np.zeros((trails, channels, imf_count, time_points))
    # the output data will be of the type: (trials, channels, num_imfs, time points)
    
    for i in range(trails):
        for j in range(channels):
            signal = input_data[i,:,j]
            imf_val = emd(signal)
            
            #For safety reasons, we will pad with zeros incase the number of IMFs obtained are lesser than what we need
            
            if len(imf_val) < imf_count:
                imf_padded = np.pad(imf_val, ((0, imf_count - len(imf_val)), (0,0)), mode = 'constant')
            else:
                imf_padded = imf_val[:imf_count]
            
            imfs[i,j] = imf_padded
    return imfs

In [ ]:
a_s = emd_signals(np.array(all_a_s))
b_s = emd_signals(np.array(all_b_s))

# Feature Extraction

### EMD Features

In [131]:
def emd_features(imfs):
    #We take the imf data for all the signals and get the statisitcal features which are:
    # mean, variance, energy and entropy.
    # these may seem random but all of them are generally used along with skewness and kurtosis for effective feature extraction in non stationary signals
    #The input data is of the form: (trials, channels, num_imfs, time points)
    
    trials, channels, num_imfs, time_points = imfs.shape
    features = []
    
    for i in range(trials):
        trail_feature = []
        for j in range(channels):
            for k in range(num_imfs):
                imf = imfs[i,j,k]
                
                energy = np.sum(imf**2)
                entropy = -np.sum(imf * np.log2(np.abs(imf) + 1e-12))
                mean = np.mean(imf)
                variance = np.var(imf)
                
                trail_feature.extend([energy,entropy,mean,variance])
        
        features.append(trail_feature)
        
    return np.array(features)
                

In [ ]:
features_a = emd_features(a_s)
features_b = emd_features(b_s)

# Combining Features

In [150]:
X = np.vstack((features_a,features_b))

y = np.array([0]*len(features_a) + [1]*len(features_b))

# Train and Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
with open("X_train_ML", 'wb') as fp:
    pickle.dump(X_train, fp)

with open("X_test_ML", 'wb') as fp:
    pickle.dump(X_test, fp)

with open("y_train_ML", 'wb') as fp:
    pickle.dump(y_train, fp)

with open("y_test_ML", 'wb') as fp:
    pickle.dump(y_test, fp)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model Training

### SVM

In [154]:
model_svm = svm.SVC()
model_svm.fit(X_train, y_train)
model_pkl_file = "SVM_MODEL.pkl"  

# with open(model_pkl_file, 'wb') as file:  
#     pickle.dump(model_svm, file)

In [ ]:
# Make predictions on the test set
y_pred = model_svm.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred)) 
conf_mat = confusion_matrix(y_test, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = [0, 1])
cm_display.plot()
plt.show()

### Random Forest

In [156]:
model_forest = RandomForestClassifier(max_depth=2, random_state=0)
model_forest.fit(X_train, y_train)
model_pkl_file = "RANDOM_FOREST_MODEL.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_svm, file)

In [ ]:
# Make predictions on the test set
y_pred = model_forest.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred)) 

conf_mat = confusion_matrix(y_test, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = [0, 1])
cm_display.plot()
plt.show()